<div class="labtitle">Projet : résolution de l'équation de la chaleur en 2D sur un rectange </div>
<div class="labauthor">This lab has been created by Pr. [Michaël Krajecki](mailto:michael.krajecki@univ-reims.fr) (University of Reims, CReSTIC and ROMEO HPC Centre)
</div>

Follow [@KrajeckiMichael](https://twitter.com/@krajeckimichael) and [@HPCromeo](https://twitter.com/@hpcromeo) on Twitter.

# Objectifs du projet

Ce projet rentre dans le cadre de l'évaluation de l'EC CHPS0811. 

Vous réaliserez l'ensemble des développements et des expérimentations par l'intermédiare de ce notebook qui sera le seul élément pris en compte pour l'évaluation.

A la fin du semestre, votre travail fera l'objet d'une présentation orale.

Vous travaillerez en binôme. Chaque binôme m’enverra un email avant le **1er mars, délai de rigueur,** pour m’informer de sa constitution. 

![EquationChaleur2D](http://python-prepa.github.io/_images/plot_edp5_2D_heat_vect_1.png)
Source de l'image : [ENS Paris](http://python-prepa.github.io/edp_chaleur.html)

Votre objectif est de produire un **code C CUDA qui proposera une résolution efficace sur 1 GPU** de l'équation de la chaleur en 2D sur un rectange.

Pour cela, vous avez à votre disposition quelques liens internet qui vous présente :
- le problème et sa résolution mathématique ;
- les approches classiques de résolution informatique.

En pratique, vous proposerez :
1. Un code C séquentiel de résolution de l'équation en 2D
2. Un code C OpenMP de résolution parallèle
3. Un code C CUDA de résolution de l'équation sur 1 GPU
4. Vous menerez une évaluation des performances entre ces différentes versions en tenant compte (a) du nombre de coeurs de calcul utilisés (b) de la taille du problème (c) des choix de répertations des calculs sur GPU
5. En vous appuyant sur les exemples fournis dans le lab *DocumentationROMEOLAB* vous proposerez une animation afin de visualiser la diffusion de la chaleur
6. De façon optionnelle, vous pourrez prosposer une résolution multi-GPU

**Mardi 26 février**, en début de cours, je répondrai à vos questions.

## Version de cuda et du compilateur Intel

Vous travaillerez uniquement avec la version [CUDA 9.0](https://docs.nvidia.com/cuda/archive/9.0/) et le compilateur [Intel 2017](https://software.intel.com/en-us/c-compilers/documentation/view-all)

Les cellules suivantes vous permettent de vérifier que les compilateurs et les ressources matérielles sont disponibles dans ce notebook.


In [ ]:
%%bash
echo $HOSTNAME
echo "Version du compilateur Intel C++"
icc --version

In [ ]:
%%bash
echo $HOSTNAME
echo "Version du compilateur NVIDIA CUDA"
nvcc --version

In [ ]:
%%bash
echo $HOSTNAME
nvidia-smi

## L'équation de la chaleur ?

Dans cette section, vous pourrez parcourir quelques liens internet qui vous permettront de comprendre l'équation de la chaleur.

Dans un premier temps, je vous conseille de regarder la vidéo de Christophe Finot qui décrit le principe de la diffusion de la chaleur en une dimension (sur une barre).


In [ ]:
from IPython.display import IFrame
IFrame('https://www.youtube.com/embed/EzAlblWb0jg', width=960, height=720)



Sur [TangenteX](http://tangentex.com/EquationChaleur.htm), vous obtiendrez :
* une description mathématique du problème
* des algorithmes de résolution

Je vous conseille également de parcourir cette [page](http://robert.mellet.pagesperso-orange.fr/diff/diff_01.htm) extraite [du pont Bachelier](http://robert.mellet.pagesperso-orange.fr/index.htm).

## Une implémentation multi-GPU proposée par NVIDIA

Maintenant que vous avez compris les principes de la résolution de l'équation de la chaleur en 2D, vous pourrez découvrir une présentation faite par [Jiri Kraus lors de GTC2018](http://on-demand-gtc.gputechconf.com/gtc-quicklink/eCVNLP6) qui vous présente comment il a résolu ce problème sur un serveur multi-GPU.

Bien évidemment, il ne s'agit pas pour vous de reproduire exactement son code, mais plutôt de vous en inspirer pour réaliser votre propre implémentation.

## Votre projet
A partir de ce point du lab, vous pouvez ajouter toutes les cellules que vous souhaitez pour mener à bien votre projet.
En particulier, vous décrirez :
* l'algorithme que vous avez mis en oeuvre
* les différentes implémentations que vous avez réalisées
* l'analyse expérimentale que vous avez menée

Vous définirez une section pour chaque partie et version du code.

Bon courage à tous

# Réalisation
## Préparation du code

In [ ]:
%%bash
# extract archive to update files (no git possible)
mkdir -p code/build
cd code/ && rm -rf *
cd ../
tar xvf progGpu.tar.gz -C code/
cd code/build && rm -rf *
module load cmake/3.9.1 opencv/3.2.0
cd ../
# génération du Makefile
./generateCmake.sh

## Exemple séquentiel

In [ ]:
%%bash
# compilation
module load cmake/3.9.1 opencv/3.2.0 gcc/7.2.0
cd code/
./clean.sh
./build.sh Prog
OMP_NUM_THREADS=1 time ./prog config.conf

Scanning dependencies of target Prog
[ 20%] Building C object CMakeFiles/Prog.dir/omp_seq_code/mainOMP.c.o
[ 40%] Building C object CMakeFiles/Prog.dir/omp_seq_code/run_functions.c.o
[ 60%] Building C object CMakeFiles/Prog.dir/omp_seq_code/compute_functions.c.o
[ 80%] Building C object CMakeFiles/Prog.dir/omp_seq_code/images.c.o
[100%] Linking C executable Prog
[100%] Built target Prog


In [ ]:
import os
from IPython.display import Image, display
from ipywidgets import interact, IntSlider
def display_result(rep):
    imgs = sorted([i for i in os.listdir(rep) if i.find('.png')>0])
    def loadimg(image):
        display(Image(open(os.path.join(rep, imgs[image]),'rb').read()))
        print(imgs[image])
    kslider = IntSlider(min=0,max=len(imgs)-1,step=1,value=3)
    return interact(loadimg ,image=kslider)
display_result('code/images');

## Exemple OpenMP

In [ ]:
%%bash
# compilation
module load cmake/3.9.1 opencv/3.2.0 gcc/7.2.0
cd code/
./clean.sh
./build.sh Prog
time ./prog config.conf

In [ ]:
import os
from IPython.display import Image, display
from ipywidgets import interact, IntSlider
def display_result(rep):
    imgs = sorted([i for i in os.listdir(rep) if i.find('.png')>0])
    def loadimg(image):
        display(Image(open(os.path.join(rep, imgs[image]),'rb').read()))
        print(imgs[image])
    kslider = IntSlider(min=0,max=len(imgs)-1,step=1,value=3)
    return interact(loadimg ,image=kslider)
display_result('code/images');

In [ ]:
%%bash
# clean pictures
cd code/images
rm *.png

## Exemple CUDA

In [ ]:
%%bash
module load cmake/3.9.1 opencv/3.2.0
cd code/
./clean.sh
./build.sh ProgGPU

In [ ]:
%%bash
module load gcc/7.2.0
cd code/
cuda-memcheck prog config.conf

### Modélisation
On reprend le même code que pour OpenMP/séquentiel (en enlevant les pragmas, les boucles for, etc.), on ajoute ensuite les identifiants de blocs, etc…

Le calcul des dimensions du kernel n'a pas de justification : le code CUDA présente des erreurs avec le contrôle de cuda-memcheck.

# Conclusion
Les grosses difficultés étaient : de configurer correctement mon CMakeLists, d'essayer de comprendre le sujet et l'adapter.

Au final, j'ai choisi la version du lab d'OpenACC. Aussi, s'il était possible de faire une version OpenACC du projet, je pense que nous aurions réussi beaucoup plus facilement et rapidement à tirer de la performance du code, notamment avec la mémoire partagée des threads (directive cache pour OpenAcc il me semble)